# Surveying Telescopes

Analysing data from an optical interferometer to determine the 3-dimensional locations of the telescopes.

Data are a complete year (2012) of optical path delay observations from the CHARA interferometer in California (see http://www.chara.gsu.edu).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('2012_S1W2.csv')
df

,bas1,h,m,s,star,az,el,delay1,delay2
0,D/S1W2,4,28,41.0,#218703,45.783151,77.837817,30.208084,57.975419
1,D/S1W2,4,43,36.0,#254159,43.713066,63.042100,30.208158,82.753579
2,D/S1W2,4,49,43.0,#254159,42.724253,63.910400,30.208084,82.868638
3,D/S1W2,5,6,25.0,#236884,43.055678,76.061440,20.208139,52.840598
4,D/S1W2,5,14,3.0,#254159,37.675613,67.175503,20.208139,73.915214
5,D/S1W2,5,20,16.0,#254193,36.084651,67.947418,20.208139,74.302185
6,D/S1W2,5,29,38.0,#250761,53.854249,75.161283,20.208139,44.938684
7,D/S1W2,5,36,7.0,#254159,31.210791,69.772064,20.208139,75.670124
8,D/S1W2,5,42,27.0,#254193,28.998253,70.431206,20.208139,76.281180
9,D/S1W2,5,52,32.0,#245262,11.066807,69.283330,0.208100,73.612800


In [7]:
# Single Value Decomposition method

# the design matrix
A = np.ones((len(df),4))
A[:,0] = np.cos(df['el']*np.pi/180.0)*np.sin(df['az']*np.pi/180.0)
A[:,1] = np.cos(df['el']*np.pi/180.0)*np.cos(df['az']*np.pi/180.0)
A[:,2] = np.sin(df['el']*np.pi/180.0)

# the data matrix
b = np.array(df['delay2'])-np.array(df['delay1'])

# parameter fitting
Ainv = np.linalg.pinv(A)
x = np.dot(Ainv,np.transpose(b))
print(x)

[-69.09506917 199.3367632    0.45754536   8.46702096]
